In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import sys
import os

sys.path.append(os.path.join(os.path.abspath(""), ".."))

In [5]:
sys.path.append(os.path.join(os.path.abspath(""), ".."))

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

clickbait = pd.read_csv("../data/clickbait_data.csv")

X, y = clickbait.headline.values, clickbait.clickbait.values

X_train, X_rest, y_train, y_rest = train_test_split(X, y, test_size=.4)

X_test, X_valid, y_test, y_valid = train_test_split(
    X_rest, y_rest, test_size=.5)

X_valid, X_dev, y_valid, y_dev = train_test_split(
    X_valid, y_valid, test_size=.5)

print(f"train size: {len(X_train)}")
print(f"test size: {len(X_test)}")
print(f"validation size: {len(X_valid)}")
print(f"dev size: {len(X_dev)}")

train size: 19200
test size: 6400
validation size: 3200
dev size: 3200


In [17]:
from sentence_transformers import SentenceTransformer, models
from sentence_transformers import datasets, util, losses, evaluation
from torch.utils.data import DataLoader


# tsdae finetuning

model_name = "bert-base-uncased"
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(
    word_embedding_model.get_word_embedding_dimension(), "cls")

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# Create the special denoising dataset that adds noise on-the-fly
dev_dataset = datasets.DenoisingAutoEncoderDataset(X_dev)

# DataLoader to batch your data
dev_dataloader = DataLoader(dev_dataset, batch_size=8, shuffle=True)

# Use the denoising auto-encoder loss
dev_loss = losses.DenoisingAutoEncoderLoss(
    model,
    decoder_name_or_path=model_name,
    tie_encoder_decoder=True)

# all the fit method
model.fit(
    train_objectives=[(dev_dataloader, dev_loss)],
    epochs=1,
    weight_decay=0,
    scheduler="constantlr",
    optimizer_params={"lr": 3e-5},
    show_progress_bar=True
)

os.makedirs("../output/", exist_ok=True)
model.save("output/tsdae-model")


Downloading: 100%|██████████| 570/570 [00:00<00:00, 290kB/s]
Downloading: 100%|██████████| 440M/440M [01:38<00:00, 4.47MB/s]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Downloading: 100%|██████████| 

KeyboardInterrupt: 

In [ ]:
from sentence_transformers import SentencesDataset
from sentence_transformers.readers import InputExample


# set up train dataset
train_examples = [
    InputExample(texts=[s], label=l) for s, l in zip(X_train, y_train)]
train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)
train_loss = losses.BatchHardTripletLoss(model=model)

# need proper hyper params
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=5,
    weight_decay=0,
    # scheduler="constantlr",
    # optimizer_params={"lr": 3e-5},
    show_progress_bar=True
)

# add prediction / eval code here